<a href="https://githubtocolab.com/kaust-halo/geeet/blob/master/examples/notebooks/01_geeet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Introduction to geeet

*geeet* is designed to run ET models (and some of their components) either locally or within an Earth Engine environment. 

This notebook demonstrates this hybrid nature by running a minimum working (toy) example. 

## On-premises data 

You can run ET models in geeet directly using numerical data given as a list of values, a [numpy ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html),  or [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html) (including [lazy](https://docs.xarray.dev/en/stable/user-guide/terminology.html#term-lazy) evaluation as well).

### Numpy array

The simplest model run is using a list of numerical values given to the ET model as keyword arguments. The following example runs the model with inputs as lists or numpy arrays. The returned output is the same in both cases: a dictionary of numpy arrays. 

In [70]:
import numpy as np
import geeet

list_inputs = dict(
    Tr = [295, 295],    # Radiometric temperature (K)
    Alb = [0.2, 0.2],   # Albedo (-)
    NDVI = [0.8, 0.8],  # NDVI (-)
    P = [95500, 95500], # Surface pressure (Pa)
    Ta = [293, 293],    # Air temperature (K)
    U = [5,5],          # Wind speed (m/s)
    Sdn = [800, 400],   # Shortwave downward radiation (W/m²)
    Ldn = [300, 200]    # Longwave downward radiation (W/m²)
)
np_inputs = {key:np.array(value) for key,value in list_inputs.items()}

scalar_inputs = dict(
    doy = 1,            # Day of year
    time = 11,          # Local hour
    Vza = 0,            # Viewing zenith angle (degrees)
    longitude = 38.25,  # Longitude (degrees)
    latitude = 30,      # Latitude (degrees)
    zU = 10,            # Wind measurement height (m)
    zT = 2              # Temperature measurement height (m)
)

et_list = geeet.tseb.tseb_series(**list_inputs, **scalar_inputs)   # This
et_np = geeet.tseb.tseb_series(**np_inputs, **scalar_inputs)       # and this result are equal.
f = lambda s,k,a,b,p: f'{s} {np.array2string(np.array(a[k]), precision=p)} | {np.array2string(np.array(b[k]), precision=p)}'
print('Energy balance components                         et_list           et_np')
print(f('Net radiation (W/m²):                          ', "Rn", et_list, et_np, 1))
print(f('Net radiation (W/m²) from canopy source:       ', "Rnc", et_list, et_np, 1))
print(f('Net radiation (W/m²) from soil source:         ', "Rns", et_list, et_np, 1))
print(f('Latent heat flux (W/m²):                       ', "LE", et_list, et_np, 1))
print(f('Latent heat flux (W/m²) from canopy source:    ', "LEc", et_list, et_np, 1))
print(f('Latent heat flux (W/m²) from soil source:      ', "LEs", et_list, et_np, 1))
print(f('Sensible heat flux (W/m²) from canopy source:  ', "Hc", et_list, et_np, 1))
print(f('Sensible heat flux (W/m²) from soil source:    ', "Hs", et_list, et_np, 1))
print(f('Ground heat flux (W/m²):                       ', "G", et_list, et_np, 1))

Energy balance components                         et_list           et_np
Net radiation (W/m²):                           [524.5 104.5] | [524.5 104.5]
Net radiation (W/m²) from canopy source:        [366.7  73.1] | [366.7  73.1]
Net radiation (W/m²) from soil source:          [157.8  31.4] | [157.8  31.4]
Latent heat flux (W/m²):                        [405.5  88.5] | [405.5  88.5]
Latent heat flux (W/m²) from canopy source:     [320.1  63.3] | [320.1  63.3]
Latent heat flux (W/m²) from soil source:       [85.4 25.2] | [85.4 25.2]
Sensible heat flux (W/m²) from canopy source:   [46.7  9.8] | [46.7  9.8]
Sensible heat flux (W/m²) from soil source:     [28.5 -2.5] | [28.5 -2.5]
Ground heat flux (W/m²):                        [43.9  8.7] | [43.9  8.7]


### xarray

The following example shows the same model run but with the inputs given as a single `xarray.Dataset`. Note that each input requires a name that is different to the short keyword argument from the previous example. Consult the documentation for the ET model to see the input band names.   

> You will need to install `xarray` if you don't already have it for this cell to work. 

In this case the output is returned as the input `xarray.Dataset` with additional data variables.

In [71]:
import xarray as xr

xr_inputs = xr.merge([
                xr.DataArray(list_inputs["Alb"]).rename("albedo"),
                xr.DataArray(list_inputs["NDVI"]).rename("NDVI"),
                xr.DataArray(list_inputs["Tr"]).rename("radiometric_temperature"),
                xr.DataArray(list_inputs["Ta"]).rename("air_temperature"),
                xr.DataArray(list_inputs["P"]).rename("surface_pressure"),
                xr.DataArray(list_inputs["U"]).rename("wind_speed"),
                xr.DataArray(list_inputs["Sdn"]).rename("solar_radiation"),
                xr.DataArray(list_inputs["Ldn"]).rename("thermal_radiation"),
            ])
            
et_xr = geeet.tseb.tseb_series(xr_inputs, **scalar_inputs)
et_xr

<xarray.Dataset> Size: 384B
Dimensions:                  (dim_0: 2)
Dimensions without coordinates: dim_0
Data variables: (12/24)
    albedo                   (dim_0) float64 16B 0.2 0.2
    NDVI                     (dim_0) float64 16B 0.8 0.8
    radiometric_temperature  (dim_0) int64 16B 295 295
    air_temperature          (dim_0) int64 16B 293 293
    surface_pressure         (dim_0) int64 16B 95500 95500
    wind_speed               (dim_0) int64 16B 5 5
    ...                       ...
    Tc                       (dim_0) float64 16B 294.7 295.2
    Tac                      (dim_0) float64 16B 296.0 294.5
    ra                       (dim_0) float64 16B 40.87 297.9
    rs                       (dim_0) float64 16B 79.94 192.4
    rx                       (dim_0) float64 16B 11.68 31.11
    it                       (dim_0) float64 16B 0.0 2.0

## Earth Engine 

### ee.Image

The ET model run for a single `ee.Image` is similar to the `xarray.Dataset` model run, in the sense that the inputs are given as a dataset (here as a multi-band *ee.Image*), and the outputs are added as additional variables (here bands). 

> For this next example, you will need the `earthengine-api` and an authenticated EE account. 

#### Preparing the dataset

Let's map the values used in the examples above to two separate constant multi-band images, using the same names given as in the [xarray](#xarray) example. We also set the `scalar_inputs` as properties in the image. 

In [72]:
import ee
ee.Initialize()

# Transform the list_inputs dictionary to lists of dictionaries
list_inputs_t = []
for i in range(2):
    list_inputs_t.append(
        {key: value[i] for key, value in list_inputs.items()}
    )
# Separate constant ee.Images:
ee_images = []
for x in list_inputs_t:
    ee_images.append(
        ee.Image(ee.Dictionary(x).toImage())
        # Rename (same names as in the xarray example)
        .select(["Alb", "NDVI", "Tr", "Ta", "P", "U", "Sdn", "Ldn"],
                ["albedo",
                "NDVI",
                "radiometric_temperature",
                "air_temperature",
                "surface_pressure",
                "wind_speed",
                "solar_radiation",
                "thermal_radiation"])
        .set({**scalar_inputs,**{"viewing_zenith": scalar_inputs["Vza"]}})
    )
ee_image_collection = ee.ImageCollection(ee_images)  # We will also prepare an ee.ImageCollection for demonstrating how to map the ET model. 

#### Single image ET model

It's as simple as using the same ET model but with the `ee.Image`:

In [73]:
et_ee = geeet.tseb.tseb_series(ee_images[0])

The output is a `ee.Image` with the input and output bands:

In [74]:
et_ee.bandNames().getInfo()

['albedo',
 'NDVI',
 'radiometric_temperature',
 'air_temperature',
 'surface_pressure',
 'wind_speed',
 'solar_radiation',
 'thermal_radiation',
 'Tc',
 'Ts',
 'Tac',
 'Hc',
 'Hs',
 'LEc',
 'LEs',
 'Ra',
 'Rs',
 'Rx',
 'Ustar',
 'alphaPT',
 'iteration',
 'LE',
 'H',
 'G',
 'Rn',
 'Rns',
 'Rnc']

To actually run the model in EE, let's define a helper function to return the value in a single point geometry. 

In [75]:
def pt_reducer(img):
    return img.reduceRegion(reducer=ee.Reducer.mean(), 
                               geometry=dict(type="Point", 
                               coordinates=[
                                   scalar_inputs["longitude"], 
                                   scalar_inputs["latitude"]
                                ]),
                               scale=30)

Now let's request the computation from the EE interactive environment (synchronous result), then let's compare the energy balance components with the numpy array results. 

In [76]:
et_ee_pt = pt_reducer(et_ee).getInfo()  # This waits for the result from EE interactive environment. 

L = lambda s,k,a,b: f'{s} {a[k][0]:.2f} | {b[k]:.2f}'  # a: the np array result (list of 2 pts); b: our single ee pt
print(L('Net radiation (W/m²):                          ', "Rn", et_np, et_ee_pt))
print(L('Net radiation (W/m²) from canopy source:       ', "Rnc",et_np, et_ee_pt))
print(L('Net radiation (W/m²) from soil source:         ', "Rns",et_np, et_ee_pt))
print(L('Latent heat flux (W/m²):                       ', "LE", et_np, et_ee_pt))
print(L('Latent heat flux (W/m²) from canopy source:    ', "LEc",et_np, et_ee_pt))
print(L('Latent heat flux (W/m²) from soil source:      ', "LEs",et_np, et_ee_pt))
print(L('Sensible heat flux (W/m²) from canopy source:  ', "Hc", et_np, et_ee_pt))
print(L('Sensible heat flux (W/m²) from soil source:    ', "Hs", et_np, et_ee_pt))
print(L('Ground heat flux (W/m²):                       ', "G",  et_np, et_ee_pt))

Net radiation (W/m²):                           524.52 | 524.52
Net radiation (W/m²) from canopy source:        366.72 | 366.72
Net radiation (W/m²) from soil source:          157.80 | 157.80
Latent heat flux (W/m²):                        405.50 | 405.50
Latent heat flux (W/m²) from canopy source:     320.06 | 320.06
Latent heat flux (W/m²) from soil source:       85.44 | 85.44
Sensible heat flux (W/m²) from canopy source:   46.65 | 46.65
Sensible heat flux (W/m²) from soil source:     28.45 | 28.45
Ground heat flux (W/m²):                        43.91 | 43.91


#### Image collection

For an image collection, it's easy to map the ET model to the collection:

In [77]:
et_ee_collection = ee_image_collection.map(geeet.tseb.tseb_series)

To reduce the values from the image collection, use the `image_collection` helper function from the `geeet.eepredefined.reducers` module. 

This is a more flexible wrapper to `ee.Image.reduceRegions` that allows mapping some bands to `ee.Reducer.mean` and optionally other bands to `ee.Reducer.sum`. 

It requires a `ee.FeatureCollection` as the regions to apply the reducer(s), and of course the image collection. Here, let's define a simple feature collection consisting of a single point. 

In [79]:
feature_collection = dict(type="FeatureCollection", features=[
    dict(type="Feature", properties=dict(name="my point"),
         geometry=dict(type="Point", 
        coordinates = [scalar_inputs["longitude"], 
                      scalar_inputs["latitude"]])
)]) 

Now let's use it to reduce our image collection:

In [81]:
from geeet.eepredefined import reducers

bands = ["LE", "LEs", "LEc", "Hs", "Hc", "Rn", "Rnc", "Rns", "G"]

reduced = reducers.image_collection(feature_collection,
    img_collection = ee.ImageCollection(et_ee_collection  # The image collection to reduce. Here our two-image collection
    # However, the reducers module requires that each image have a system:time_start, so let's put one here:
                    .map(lambda img: img.set({"system:time_start": 1704096000000})),
    ), 
    mean_bands = bands,  # The bands to reduce using ee.Reudcer.mean
    feature_properties = ["name"] # Optionally keep properties from each feature. 
) # This is a ee.FeatureCollection

# For easier comparison with the numpy array aexample, let's 
# request the output as a dictionary of arrays:
band_agg_dict = {band: reduced.aggregate_array(band) for band in bands}
ee_outputs = ee.Dictionary(band_agg_dict).getInfo()

# Finally, let's compare these with the on-premises results:
print('Energy balance components                         np_outputs           ee_outputs')
print(f('Net radiation (W/m²):                          ', "Rn",  et_np, ee_outputs, 1))
print(f('Net radiation (W/m²) from canopy source:       ', "Rnc", et_np, ee_outputs, 1))
print(f('Net radiation (W/m²) from soil source:         ', "Rns", et_np, ee_outputs, 1))
print(f('Latent heat flux (W/m²):                       ', "LE",  et_np, ee_outputs, 1))
print(f('Latent heat flux (W/m²) from canopy source:    ', "LEc", et_np, ee_outputs, 1))
print(f('Latent heat flux (W/m²) from soil source:      ', "LEs", et_np, ee_outputs, 1))
print(f('Sensible heat flux (W/m²) from canopy source:  ', "Hc",  et_np, ee_outputs, 1))
print(f('Sensible heat flux (W/m²) from soil source:    ', "Hs",  et_np, ee_outputs, 1))
print(f('Ground heat flux (W/m²):                       ', "G",   et_np, ee_outputs, 1))

Energy balance components                         np_outputs           ee_outputs
Net radiation (W/m²):                           [524.5 104.5] | [524.5 104.5]
Net radiation (W/m²) from canopy source:        [366.7  73.1] | [366.7  73.1]
Net radiation (W/m²) from soil source:          [157.8  31.4] | [157.8  31.4]
Latent heat flux (W/m²):                        [405.5  88.5] | [405.5  88.5]
Latent heat flux (W/m²) from canopy source:     [320.1  63.3] | [320.1  63.3]
Latent heat flux (W/m²) from soil source:       [85.4 25.2] | [85.4 25.2]
Sensible heat flux (W/m²) from canopy source:   [46.7  9.8] | [46.7  9.8]
Sensible heat flux (W/m²) from soil source:     [28.5 -2.5] | [28.5 -2.5]
Ground heat flux (W/m²):                        [43.9  8.7] | [43.9  8.7]
